In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import syft as sy
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/opt/conda/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = 2
        self.lr = 0.001
        self.log_interval = 100

args = Arguments()

In our setting, we assume that the server has access to some data to first train its model. Here is the MNIST training set.

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

SMPC uses crypto protocols which require to work on integers. We leverage here the PySyft tensor abstraction to convert PyTorch Float tensors into Fixed Precision Tensors using .fix_precision(). For example 0.123 with precision 2 does a rounding at the 2nd decimal digit so the number stored is the integer 12.

Second, the client has some data and would like to have predictions on it using the server's model. This client encrypts its data by sharing it additively across two workers alice and bob.

In [6]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_precision().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
    ))

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [8]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

In [15]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.311289
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.549812
Train Epoch: 1 [12800/60032 (21%)]	Loss: 0.256326
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.232705
Train Epoch: 1 [25600/60032 (43%)]	Loss: 0.134745
Train Epoch: 1 [32000/60032 (53%)]	Loss: 0.202663
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.189081
Train Epoch: 1 [44800/60032 (75%)]	Loss: 0.159619
Train Epoch: 1 [51200/60032 (85%)]	Loss: 0.216645
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.088952
Train Epoch: 2 [0/60032 (0%)]	Loss: 0.072995
Train Epoch: 2 [6400/60032 (11%)]	Loss: 0.195817
Train Epoch: 2 [12800/60032 (21%)]	Loss: 0.038129
Train Epoch: 2 [19200/60032 (32%)]	Loss: 0.172233
Train Epoch: 2 [25600/60032 (43%)]	Loss: 0.027773
Train Epoch: 2 [32000/60032 (53%)]	Loss: 0.135632
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.020831
Train Epoch: 2 [44800/60032 (75%)]	Loss: 0.048880
Train Epoch: 2 [51200/60032 (85%)]	Loss: 0.139644
Train Epoch: 2 [57600/60032 (96%)]	Loss: 0.093562
Train Epoch:

In [16]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            output = F.log_softmax(output, dim=1)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:
test(args, model, test_loader)


Test set: Average loss: 0.0760, Accuracy: 9772/10000 (98%)



Our model is now trained and ready to be provided as a service!

Now, as the server, we send the model to the workers holding the data. Because the model is sensitive information (you've spent time optimizing it!), you don't want to disclose its weights so you secret share the model just like we did with the dataset earlier.

In [18]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [21]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    n_test_batches = 10
    with torch.no_grad():
        for data, target in test_loader[:n_test_batches]:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
# This 'test' function performs the encrypted evaluation. The model weights, the data inputs, the prediction and the target used for scoring are all encrypted!

# However as you can observe, the syntax is very similar to normal PyTorch testing! Nice!

# The only thing we decrypt from the server side is the final score at the end of our 200 items batches to verify predictions were on average good.      
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))

In [22]:
test(args, model, private_test_loader)

Test set: Accuracy: 46/50 (92%)
Test set: Accuracy: 94/100 (94%)
Test set: Accuracy: 143/150 (95%)
Test set: Accuracy: 190/200 (95%)
Test set: Accuracy: 239/250 (96%)
Test set: Accuracy: 287/300 (96%)
Test set: Accuracy: 336/350 (96%)
Test set: Accuracy: 386/400 (96%)
Test set: Accuracy: 435/450 (97%)
Test set: Accuracy: 484/500 (97%)
